In [ ]:
// Core ingredient recommendation interface
interface IngredientRecommendation {
  name: string;
  type: string;
  category?: string;
  elementalProperties?: ElementalProperties;
  qualities?: string[];
  matchScore: number;
  modality?: Modality;
  recommendations?: string[];
  description?: string;
  totalScore?: number;
  elementalScore?: number;
  astrologicalScore?: number;
  seasonalScore?: number;
  dietary?: string[];
  // Extended properties
  flavorProfile?: Record<string, number>;
  cuisine?: string;
  sensoryProfile?: {
    taste: Record<string, number>;
    aroma: Record<string, number>;
    texture: Record<string, number>;
  };
  recommendedCookingMethods?: CookingMethod[];
  pairingRecommendations?: {
    complementary: string[];
    contrasting: string[];
  };
}

// Grouped recommendations by category
interface GroupedIngredientRecommendations {
  vegetables?: IngredientRecommendation[];
  fruits?: IngredientRecommendation[];
  proteins?: IngredientRecommendation[];
  grains?: IngredientRecommendation[];
  spices?: IngredientRecommendation[];
  herbs?: IngredientRecommendation[];
  [key: string]: IngredientRecommendation[] | undefined;
}

// Recommendation options and filters
interface RecommendationOptions {
  currentSeason?: string;
  dietaryPreferences?: string[];
  modalityPreference?: Modality;
  currentZodiac?: string;
  limit?: number;
  excludeIngredients?: string[];
  includeOnly?: string[];
  category?: string;
}


In [ ]:
/**
 * Primary function to get recommended ingredients based on astrological state
 * Filters ingredients by planetary rulers and applies elemental scoring
 */
function getRecommendedIngredients(astroState: AstrologicalState): EnhancedIngredient[] {
  // 1. Extract active planets from astrological state
  const activePlanets = astroState.activePlanets || [];
  
  // 2. Filter ingredients by planetary rulers
  let filteredIngredients = allIngredients.filter(ingredient => {
    const rulingPlanets = ingredient?.astrologicalProfile?.rulingPlanets;
    return rulingPlanets?.some(planet => activePlanets.includes(planet));
  });
  
  // 3. Apply planetary-specific enhancements
  if (activePlanets.includes('Venus')) {
    enhanceVenusIngredientBatch(filteredIngredients, astroState);
  }
  if (activePlanets.includes('Mars')) {
    enhanceMarsIngredientScoring(filteredIngredients, astroState);
  }
  if (activePlanets.includes('Mercury')) {
    enhanceMercuryIngredientScoring(filteredIngredients, astroState);
  }
  
  // 4. Sort by dominant element and zodiac affinity
  // ...
  
  return filteredIngredients;
}

/**
 * Advanced recommendation function with grouping and scoring
 * Returns categorized recommendations with weighted scoring
 */
function getIngredientRecommendations(
  elementalProps: ElementalProperties & ExtendedAstrologicalData, 
  options: RecommendationOptions
): GroupedIngredientRecommendations {
  // 1. Get all ingredients and apply filters
  const allIngredients = getAllIngredients();
  
  // 2. Calculate comprehensive scoring
  const scoredIngredients = allIngredients.map(ingredient => {
    const elementalScore = calculateElementalScore(ingredient.elementalProperties, elementalProps) * 0.30;
    const modalityScore = calculateModalityScore(ingredient.qualities, options.modalityPreference) * 0.15;
    const seasonalScore = calculateSeasonalScore(ingredient, elementalProps.timestamp) * 0.15;
    const planetaryScore = calculateEnhancedPlanetaryScore(
      ingredient, 
      elementalProps.planetaryAlignment,
      planetDecans,
      rulingPlanet
    ) * 0.40;
    
    return {
      ...ingredient,
      totalScore: elementalScore + modalityScore + seasonalScore + planetaryScore,
      elementalScore,
      modalityScore,
      seasonalScore,
      planetaryScore
    };
  });
  
  // 3. Group by category and apply limits
  // ...
  
  return groupedRecommendations;
}


In [ ]:
/**
 * Calculate elemental compatibility score between ingredient and system
 * Enhanced with dominant element weighting
 */
function calculateElementalScore(
  ingredientProps?: ElementalProperties,
  systemProps?: ElementalProperties
): number {
  if (!ingredientProps || !systemProps) return 0.5;
  
  // Find dominant system element for extra weighting
  const dominantElement = Object.entries(systemProps)
    .sort((a, b) => b[1] - a[1])[0][0] as keyof ElementalProperties;
  
  let similarityScore = 0;
  let totalWeight = 0;
  
  // Process each element with similarity calculation
  for (const element of ['Fire', 'Water', 'Earth', 'Air'] as const) {
    const ingredientValue = ingredientProps[element] || 0;
    const systemValue = systemProps[element] || 0;
    
    // Calculate similarity (1 - absolute difference)
    const similarity = 1 - Math.abs(ingredientValue - systemValue);
    
    // Enhanced weighting with dominant element emphasis
    const baseWeight = systemValue + 0.25;
    const finalWeight = element === dominantElement ? baseWeight * 1.5 : baseWeight;
    
    similarityScore += similarity * finalWeight;
    totalWeight += finalWeight;
  }
  
  return totalWeight > 0 ? Math.min(1, Math.max(0, similarityScore / totalWeight)) : 0.5;
}

/**
 * Enhanced planetary scoring with decans and tarot associations
 */
function calculateEnhancedPlanetaryScore(
  ingredient: Ingredient,
  planetaryAlignment: Record<string, { sign: string; degree: number }>,
  planetDecans: Record<string, { decanNum: number, decanRuler: string, tarotCard: string }>,
  rulingPlanet: string
): number {
  if (!ingredient?.astrologicalProfile) return 0.5;
  
  let score = 0;
  let totalFactors = 0;
  
  // Check ruling planet correspondence - extra weight
  if (ingredient.astrologicalProfile.rulingPlanets?.includes(rulingPlanet)) {
    score += 1.5;
    totalFactors += 1.5;
  }
  
  // Check planetary positions and decan rulers
  Object.entries(planetaryAlignment).forEach(([planet, position]) => {
    const planetName = planet.charAt(0).toUpperCase() + planet.slice(1);
    
    // Regular planetary ruler scoring
    if (ingredient.astrologicalProfile.rulingPlanets?.includes(planetName)) {
      score += 1;
      totalFactors += 1;
    }
    
    // Sign affinities
    if (ingredient.astrologicalProfile.signAffinities?.includes(position.sign.toLowerCase())) {
      score += 1;
      totalFactors += 1;
    }
    
    // Decan ruler matches
    const decanInfo = planetDecans[planet];
    if (decanInfo && ingredient.astrologicalProfile.rulingPlanets?.includes(decanInfo.decanRuler)) {
      score += 0.8;
      totalFactors += 0.8;
    }
    
    // Tarot associations
    if (decanInfo?.tarotCard && ingredient.astrologicalProfile.tarotAssociations?.includes(decanInfo.tarotCard)) {
      score += 0.7;
      totalFactors += 0.7;
    }
  });
  
  return totalFactors === 0 ? 0.5 : Math.min(1, score / (totalFactors + 0.5));
}


In [ ]:
/**
 * Venus influence calculation - comprehensive flavor and texture analysis
 */
function calculateVenusInfluence(
  ingredient: Ingredient, 
  zodiacSign?: string,
  isVenusRetrograde = false
): number {
  let score = 0;
  
  // Base Venus association check
  if (isVenusAssociatedIngredient(ingredient.name)) {
    score += 2.0;
  }
  
  // Elemental properties alignment (Venus favors Water and Earth)
  if (ingredient.elementalProperties) {
    score += (ingredient.elementalProperties.Water || 0) * 1.5;
    score += (ingredient.elementalProperties.Earth || 0) * 1.8;
    score += (ingredient.elementalProperties.Air || 0) * 0.8;
    score += (ingredient.elementalProperties.Fire || 0) * 0.5;
  }
  
  // Flavor profile alignment (Venus loves sweet, rich, creamy)
  const flavorProfile = ingredient.flavorProfile;
  if (flavorProfile) {
    if (flavorProfile.sweet) score += flavorProfile.sweet * 2.0;
    if (flavorProfile.umami) score += flavorProfile.umami * 1.5;
    if (flavorProfile.creamy || flavorProfile.rich) {
      score += ((flavorProfile.creamy || 0) + (flavorProfile.rich || 0)) * 1.7;
    }
    if (flavorProfile.aromatic || flavorProfile.fragrant) {
      score += ((flavorProfile.aromatic || 0) + (flavorProfile.fragrant || 0)) * 1.6;
    }
  }
  
  // Zodiac sign-specific preferences
  if (zodiacSign && venusData.PlanetSpecific?.ZodiacTransit) {
    const transitData = venusData.PlanetSpecific.ZodiacTransit[zodiacSign];
    // Apply transit-specific scoring...
  }
  
  // Retrograde modifiers
  if (isVenusRetrograde) {
    // Boost preserved/dried herbs, reduce others
    if (ingredient.category === 'herb' || ingredient.preservation?.includes('dried')) {
      score *= 1.5;
    } else {
      score *= 0.8;
    }
  }
  
  return score;
}

/**
 * Mars influence calculation - focus on spicy, fiery, energetic ingredients
 */
function calculateMarsInfluence(
  ingredient: Ingredient, 
  zodiacSign?: string,
  isMarsRetrograde = false
): number {
  let score = 0;
  
  // Mars food associations
  if (marsData.FoodAssociations) {
    for (const food of marsData.FoodAssociations) {
      if (ingredient.name.toLowerCase().includes(food.toLowerCase()) || 
          food.toLowerCase().includes(ingredient.name.toLowerCase())) {
        score += 1.5;
        break;
      }
    }
  }
  
  // Elemental alignment (Mars is primarily Fire, secondarily Water)
  if (ingredient.elementalProperties) {
    score += (ingredient.elementalProperties.Fire || 0) * 1.5;
    score += (ingredient.elementalProperties.Water || 0) * 0.8;
  }
  
  // Flavor profile alignment with Mars preferences
  if (ingredient.flavorProfile) {
    for (const flavor in marsData.FlavorProfiles) {
      if (ingredient.flavorProfile[flavor]) {
        score += marsData.FlavorProfiles[flavor] * ingredient.flavorProfile[flavor];
      }
    }
  }
  
  // Zodiac-specific boosts
  if (zodiacSign && marsData.PlanetSpecific?.ZodiacTransit) {
    const transit = marsData.PlanetSpecific.ZodiacTransit[zodiacSign];
    if (transit?.Ingredients) {
      for (const transitIngredient of transit.Ingredients) {
        if (ingredient.name.toLowerCase().includes(transitIngredient.toLowerCase())) {
          score += 2.5;
          break;
        }
      }
    }
  }
  
  return score;
}

/**
 * Mercury influence - complex, varied, aromatic ingredients
 */
function calculateMercuryInfluence(
  ingredient: Ingredient, 
  zodiacSign?: string,
  isMercuryRetrograde = false
): number {
  let score = 0;
  
  // Base Mercury ruling
  if (ingredient.astrologicalProfile?.rulingPlanets?.includes('Mercury')) {
    score += 3.0;
  }
  
  // Mercury's elements (Air and Earth)
  if (ingredient.elementalProperties) {
    score += (ingredient.elementalProperties.Air || 0) * 2.0;
    score += (ingredient.elementalProperties.Earth || 0) * 1.8;
  }
  
  // Mercury qualities (aromatic, complex, stimulating)
  if (ingredient.qualities) {
    const mercuryQualityBoosts = {
      'aromatic': 1.3,
      'complex': 1.4,
      'stimulating': 1.5,
      'adaptable': 1.3,
      'versatile': 1.4
    };
    
    for (const quality of ingredient.qualities) {
      const lowerQuality = quality.toLowerCase();
      for (const [mercuryQuality, boost] of Object.entries(mercuryQualityBoosts)) {
        if (lowerQuality.includes(mercuryQuality)) {
          score += boost;
          break;
        }
      }
    }
  }
  
  // Retrograde adjustments
  if (isMercuryRetrograde) {
    if (ingredient.qualities?.includes('traditional') || ingredient.qualities?.includes('classic')) {
      score *= 1.25;
    }
    if (ingredient.qualities?.includes('exotic') || ingredient.qualities?.includes('complex')) {
      score *= 0.8;
    }
  }
  
  return score;
}


In [ ]:
/**
 * Generate ingredient recommendations based on chakra energy levels
 * Maps chakra energies to specific ingredient types and nutritional correlations
 */
function getChakraBasedRecommendations(
  chakraEnergies: ChakraEnergies,
  limit = 3
): GroupedIngredientRecommendations {
  // Find dominant chakras (highest energy levels)
  const sortedChakras = Object.entries(chakraEnergies)
    .sort(([, energyA], [, energyB]) => energyB - energyA)
    .filter(([, energy]) => energy > 0);
  
  const result: GroupedIngredientRecommendations = {};
  
  // For each significant chakra, add corresponding ingredients
  sortedChakras.forEach(([chakra, energy]) => {
    const nutritionalCorrelations = CHAKRA_NUTRITIONAL_CORRELATIONS[chakra] || [];
    const herbRecommendations = CHAKRA_HERBS[chakra] || [];
    
    // Find matching ingredients
    const matchingIngredients = allIngredients.filter(ingredient => {
      const matchesNutritional = nutritionalCorrelations.some(correlation => 
        ingredient.name?.toLowerCase().includes(correlation.toLowerCase()) || 
        ingredient.type?.toLowerCase().includes(correlation.toLowerCase())
      );
      
      const matchesHerb = herbRecommendations.some(herb => 
        ingredient.name?.toLowerCase().includes(herb.toLowerCase())
      );
      
      return matchesNutritional || matchesHerb;
    });
    
    // Add to results with chakra-based scoring
    matchingIngredients.forEach(ingredient => {
      const categoryKey = `${ingredient.type?.toLowerCase()}s` || 'others';
      
      if (!result[categoryKey]) {
        result[categoryKey] = [];
      }
      
      const recommendation: IngredientRecommendation = {
        ...ingredient,
        matchScore: energy / 10, // Normalize to 0-1 range
        recommendations: [
          `Supports ${chakra} chakra energy`,
          ...nutritionalCorrelations.filter(corr => 
            ingredient.name?.toLowerCase().includes(corr.toLowerCase())
          )
        ]
      };
      
      if (!result[categoryKey]?.some(rec => rec.name === ingredient.name)) {
        result[categoryKey]?.push(recommendation);
      }
    });
  });
  
  // Apply limit to each category
  Object.keys(result).forEach(key => {
    if (result[key]?.length > limit) {
      result[key] = result[key]?.slice(0, limit);
    }
  });
  
  return result;
}


In [ ]:
/**
 * Determine ingredient modality based on qualities and elemental properties
 * Uses hierarchical affinities: Mutability (Air > Water > Fire > Earth)
 * Fixed (Earth > Water > Fire > Air), Cardinal (Equal for all elements)
 */
function determineIngredientModality(
  qualities: string[] = [],
  elementalProperties?: ElementalProperties
): Modality {
  const normalizedQualities = qualities.map(q => q.toLowerCase());
  
  // Quality keyword matching
  const cardinalKeywords = ['initiating', 'spicy', 'pungent', 'stimulating', 'invigorating'];
  const fixedKeywords = ['grounding', 'stabilizing', 'nourishing', 'sustaining', 'foundational'];
  const mutableKeywords = ['adaptable', 'flexible', 'versatile', 'balancing', 'harmonizing'];
  
  const hasCardinalQuality = normalizedQualities.some(q => cardinalKeywords.includes(q));
  const hasFixedQuality = normalizedQualities.some(q => fixedKeywords.includes(q));
  const hasMutableQuality = normalizedQualities.some(q => mutableKeywords.includes(q));
  
  // Clear quality indicator wins
  if (hasCardinalQuality && !hasFixedQuality && !hasMutableQuality) return 'Cardinal';
  if (hasFixedQuality && !hasCardinalQuality && !hasMutableQuality) return 'Fixed';
  if (hasMutableQuality && !hasCardinalQuality && !hasFixedQuality) return 'Mutable';
  
  // Use elemental properties for determination
  if (elementalProperties) {
    const { Fire, Water, Earth, Air } = elementalProperties;
    
    // Calculate modality scores based on hierarchical affinities
    const mutableScore = (Air * 0.9) + (Water * 0.8) + (Fire * 0.7) + (Earth * 0.5);
    const fixedScore = (Earth * 0.9) + (Water * 0.8) + (Fire * 0.6) + (Air * 0.5);
    const cardinalScore = (Fire * 0.8) + (Earth * 0.8) + (Water * 0.8) + (Air * 0.8);
    
    if (mutableScore > fixedScore && mutableScore > cardinalScore) return 'Mutable';
    if (fixedScore > mutableScore && fixedScore > cardinalScore) return 'Fixed';
    return 'Cardinal';
  }
  
  return 'Mutable'; // Default
}


In [ ]:
// Example 1: Basic astrological recommendations
const astroState = {
  activePlanets: ['Venus', 'Mars'],
  zodiacSign: 'taurus',
  dominantElement: 'Earth',
  Fire: 0.2,
  Water: 0.3,
  Earth: 0.4,
  Air: 0.1,
  timestamp: new Date(),
  planetaryAlignment: {
    Venus: { sign: 'Taurus', degree: 15 },
    Mars: { sign: 'Aries', degree: 8 }
  }
};

const recommendations = getRecommendedIngredients(astroState);
console.log('Recommended ingredients:', recommendations.slice(0, 5));

// Example 2: Filtered recommendations with options
const options = {
  currentSeason: 'spring',
  dietaryPreferences: ['vegetarian'],
  modalityPreference: 'Fixed',
  limit: 10,
  category: 'vegetables'
};

const elementalProps = {
  ...astroState,
  timestamp: new Date(),
  currentStability: 0.8,
  planetaryAlignment: astroState.planetaryAlignment,
  lunarPhase: 'waxing gibbous',
  aspects: []
};

const groupedRecommendations = getIngredientRecommendations(elementalProps, options);
console.log('Grouped recommendations:', groupedRecommendations);

// Example 3: Chakra-based recommendations
const chakraEnergies = {
  root: 0.8,
  sacral: 0.6,
  solarPlexus: 0.7,
  heart: 0.9,
  throat: 0.5,
  thirdEye: 0.4,
  crown: 0.3
};

const chakraRecommendations = getChakraBasedRecommendations(chakraEnergies, 5);
console.log('Chakra-based recommendations:', chakraRecommendations);


In [ ]:
// Example: Venus in Taurus recommendations
const venusInTaurus = {
  activePlanets: ['Venus'],
  zodiacSign: 'taurus',
  isVenusRetrograde: false,
  planetaryAlignment: {
    Venus: { sign: 'Taurus', degree: 12 }
  }
};

// This would emphasize:
// - Rich, earthy flavors
// - Sensual textures
// - Traditional comfort foods
// - Root vegetables and grounding herbs

// Example: Mars in Aries recommendations  
const marsInAries = {
  activePlanets: ['Mars'],
  zodiacSign: 'aries', 
  isMarsRetrograde: false,
  planetaryAlignment: {
    Mars: { sign: 'Aries', degree: 25 }
  }
};

// This would emphasize:
// - Spicy, fiery ingredients
// - Energizing herbs and spices
// - Quick-cooking methods
// - Stimulating flavors

// Example: Mercury retrograde adjustments
const mercuryRetrograde = {
  activePlanets: ['Mercury'],
  zodiacSign: 'gemini',
  isMercuryRetrograde: true,
  planetaryAlignment: {
    Mercury: { sign: 'Gemini', degree: 18 }
  }
};

// This would emphasize:
// - Traditional, familiar ingredients
// - Simple, time-tested combinations
// - Reduced emphasis on exotic/complex flavors
// - Nostalgic food choices
